**Setup:**

In [11]:
%pip install -Uqq fastai torchvision

from fastai.vision.all import *

Note: you may need to restart the kernel to use updated packages.


Importing Data:

In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

Look at `train_data`:

In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

Show shape of `train_data`:

In [4]:
train_data.data.shape

torch.Size([60000, 28, 28])

Create Dataloaders `dl` and `valid_dl` with batch size 100:

In [5]:
dl = DataLoader(train_data, batch_size=100, shuffle=True)
valid_dl = DataLoader(test_data, batch_size=100, shuffle=True)

Import `Net` from `learning_functions.py`:

In [6]:
from learning_functions import Net
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Use GPU if available
neural_net = Net().to(device)

Define functions `train` and `test` and define a loss function and an optimizer with learning rate 0.2:

In [7]:
from torch import nn

optimizer = SGD(neural_net.parameters(), lr=0.2)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    neural_net.train()
    global optimizer
    for batch_idx, (data, target) in enumerate(dl):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = neural_net(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(dl.dataset)} ({100. * batch_idx / len(dl):.0f}%)]\t{loss.item():.6f}")

def test():
    neural_net.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in valid_dl:
            data, target = data.to(device), target.to(device)
            output = neural_net(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(valid_dl.dataset)
    print(f'\nTest Loss: Average Loss: {test_loss:.4f}, Accuracy: {correct}/{len(valid_dl.dataset)} {100. * correct / len(valid_dl.dataset):.0f}%')

Train Model for 100 epochs:

In [8]:
for epoch in range(1, 101):
        train(epoch)
        test()

/Users/preston/Projects/fastai_course/MNIST_Digit_identifier/learning_functions.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.302656
Train Epoch: 1 [2000/60000 (3%)]	2.302629
Train Epoch: 1 [4000/60000 (7%)]	2.302155
Train Epoch: 1 [6000/60000 (10%)]	2.301995
Train Epoch: 1 [8000/60000 (13%)]	2.302271
Train Epoch: 1 [10000/60000 (17%)]	2.300196
Train Epoch: 1 [12000/60000 (20%)]	2.301824
Train Epoch: 1 [14000/60000 (23%)]	2.300658
Train Epoch: 1 [16000/60000 (27%)]	2.298703
Train Epoch: 1 [18000/60000 (30%)]	2.301169
Train Epoch: 1 [20000/60000 (33%)]	2.300408
Train Epoch: 1 [22000/60000 (37%)]	2.297974
Train Epoch: 1 [24000/60000 (40%)]	2.295692
Train Epoch: 1 [26000/60000 (43%)]	2.295876
Train Epoch: 1 [28000/60000 (47%)]	2.268416
Train Epoch: 1 [30000/60000 (50%)]	2.211246
Train Epoch: 1 [32000/60000 (53%)]	2.123919
Train Epoch: 1 [34000/60000 (57%)]	2.097786
Train Epoch: 1 [36000/60000 (60%)]	2.081563
Train Epoch: 1 [38000/60000 (63%)]	1.995052
Train Epoch: 1 [40000/60000 (67%)]	2.018073
Train Epoch: 1 [42000/60000 (70%)]	1.954998
Train Epoch: 1 [44000/60000 (73%)]	1.874205

Export Predicition Class:

In [10]:
torch.save(neural_net.state_dict(), 'new_digit_identifier.pth')